<a href="https://colab.research.google.com/github/GuilhermeAlmanza/real_estate_pred/blob/master/src/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras 
import pandas as pd
import numpy as np
import tensorflow as tf

#load files

abcb4_timedata = pd.read_csv("/content/ABCB4 Dados Históricos.csv", sep=",", index_col=None)
# print(abcb4_timedata)

#preprocessing
abcb4_timedata['Data'] = pd.to_datetime(abcb4_timedata['Data'], format="%d.%m.%Y")

abcb4_timedata['Último'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Último']]
abcb4_timedata['Abertura'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Abertura']]
abcb4_timedata['Máxima'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Máxima']]
abcb4_timedata['Mínima'] = [float(record.replace(",", ".")) for record in abcb4_timedata['Mínima']]

abcb4_timedata['Vol.'] = [float(record.replace("K", "0").replace("M", "0000").replace(",", "")) for record in abcb4_timedata['Vol.']]

abcb4_timedata['Var%'] = [float(record.removesuffix("%").replace(",", "."))/100 for record in abcb4_timedata['Var%']]

df = pd.DataFrame(abcb4_timedata, index=None)
df = df.drop(columns='Data')
print(df)


      Último  Abertura  Máxima  Mínima      Vol.    Var%
0      16.45     16.62   16.70   16.39   88800.0 -0.0126
1      16.66     16.68   16.68   16.36  573200.0  0.0073
2      16.54     16.46   16.60   16.32  666300.0  0.0092
3      16.39     16.99   16.99   16.39  481500.0 -0.0261
4      16.83     17.00   17.00   16.48  840100.0 -0.0100
...      ...       ...     ...     ...       ...     ...
3835    5.91      5.76    5.91    5.76   34960.0 -0.0034
3836    5.93      5.93    5.96    5.75   34930.0 -0.0017
3837    5.94      5.79    6.11    5.79  265770.0  0.0189
3838    5.83      5.92    5.92    5.83   94940.0 -0.0068
3839    5.87      5.87    6.13    5.87  365420.0  0.0138

[3840 rows x 6 columns]


In [3]:
#Train, Test and Validation groups

from sklearn.model_selection import train_test_split

label_list_train_test = list(range(df.shape[0]))
data_train, data_test, label_train, label_test = train_test_split(df, label_list_train_test, test_size=0.25, random_state=42, shuffle=False)

# print(data_train, "\n" ,data_test)

data_validation, data_test, label_validation, label_test= train_test_split(data_test, label_test, test_size=0.66, random_state=42, shuffle=False)

print(f"data train: \n{data_train} \ndata test\n: {data_test} \ndata validation\n: {data_validation}")


data train: 
      Último  Abertura  Máxima  Mínima      Vol.    Var%
0      16.45     16.62   16.70   16.39   88800.0 -0.0126
1      16.66     16.68   16.68   16.36  573200.0  0.0073
2      16.54     16.46   16.60   16.32  666300.0  0.0092
3      16.39     16.99   16.99   16.39  481500.0 -0.0261
4      16.83     17.00   17.00   16.48  840100.0 -0.0100
...      ...       ...     ...     ...       ...     ...
2875    5.22      4.98    5.42    4.98  433550.0  0.0316
2876    5.06      5.10    5.14    4.96  473950.0 -0.0020
2877    5.07      4.92    5.08    4.86  101110.0  0.0326
2878    4.91      4.92    4.94    4.87  162590.0  0.0041
2879    4.89      4.90    4.93    4.76  367210.0  0.0041

[2880 rows x 6 columns] 
data test
:       Último  Abertura  Máxima  Mínima      Vol.    Var%
3206    6.42      6.34    6.42    6.27  211170.0  0.0239
3207    6.27      6.32    6.37    6.15  116190.0 -0.0032
3208    6.29      6.50    6.50    6.26  153770.0 -0.0323
3209    6.50      6.44    6.54    6.4

In [5]:
#Model 

import numpy as np
from keras import layers
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

num_timesteps = 1

#target data
data_train_target = (data_train.loc[:,"Último"]).values
data_train.drop(columns = 'Último')

data_test_target = (data_test.loc[:,"Último"]).values
data_test.drop(columns = 'Último')

data_validation_target = (data_validation.loc[:,"Último"]).values
data_validation.drop(columns = 'Último')

#reshape data
data_train_in = data_train.values
data_train_in = data_train_in.reshape(data_train_in.shape[0], num_timesteps, data_train_in.shape[1])

data_test_in = data_test.values
data_test_in = data_test_in.reshape(data_test_in.shape[0], num_timesteps, data_test_in.shape[1])

data_validation_in = data_validation.values
data_validation_in = data_validation_in.reshape(data_validation_in.shape[0], num_timesteps, data_validation_in.shape[1])

model = keras.Sequential([
    layers.LSTM(64, input_shape=(num_timesteps, data_train_in.shape[2])), #timesteps, features
    layers.Dense(1)
])

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
history = model.fit(data_train_in, data_train_target, epochs=50, batch_size=32, validation_data=(data_validation_in, data_validation_target))
print(history.history)
scores = model.evaluate(data_test_in, data_test_target)
print(f'scores:\nloss: {scores[0]}\nmean abs error: {scores[1]}')

predictions = model.predict(data_test_in)
print(predictions)
print(predictions.shape)

mae = round(mean_absolute_error(data_test_target, predictions) ,4)
mape = round(np.mean(np.abs((data_test_target - predictions) / data_test_target)) * 100, 4)
mse = round(mean_squared_error(data_test_target, predictions), 4)
rmse = round(mse**(0.5), 4)
rmspe = round((np.sqrt(np.mean(np.square((data_test_target - predictions) / data_test_target)))) * 100, 4)
r2 = round(r2_score(data_test_target, predictions), 4)
print(f"mae: {mae} \nmape: {mape} \nmse: {mse}, \nrmse: {rmse} \nr2: {r2}")


Epoch 1/50
90/90 [==============================] - 3s 9ms/step - loss: 133.9301 - mae: 10.7348 - val_loss: 24.5345 - val_mae: 4.8598
Epoch 2/50
90/90 [==============================] - 0s 3ms/step - loss: 106.7407 - mae: 9.3893 - val_loss: 14.6370 - val_mae: 3.7041
Epoch 3/50
90/90 [==============================] - 0s 3ms/step - loss: 86.8977 - mae: 8.2706 - val_loss: 7.7609 - val_mae: 2.6161
Epoch 4/50
90/90 [==============================] - 0s 3ms/step - loss: 70.6843 - mae: 7.2201 - val_loss: 3.5074 - val_mae: 1.6377
Epoch 5/50
90/90 [==============================] - 0s 3ms/step - loss: 57.6301 - mae: 6.2737 - val_loss: 1.3862 - val_mae: 0.9087
Epoch 6/50
90/90 [==============================] - 0s 3ms/step - loss: 47.3017 - mae: 5.4917 - val_loss: 0.9456 - val_mae: 0.7868
Epoch 7/50
90/90 [==============================] - 0s 3ms/step - loss: 39.2778 - mae: 4.9108 - val_loss: 1.8016 - val_mae: 1.1491
Epoch 8/50
90/90 [==============================] - 0s 3ms/step - loss: 33.160